In [31]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

In [32]:
import pandas as pd

In [33]:
challenger_data = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/challenger.csv")

In [34]:
challenger_data_ = challenger_data.values

In [40]:
temperature_ = challenger_data_[:, 0]
temperature_ = tf.convert_to_tensor(temperature_, dtype=tf.float32)
D_ = challenger_data_[:, 1]       
D = tf.convert_to_tensor(D_, dtype=tf.float32)

In [43]:
beta = tfd.Normal(name="beta", loc=0.3, scale=1000.).sample()
alpha = tfd.Normal(name="alpha", loc=-15., scale=1000.).sample()
p_deterministic = tfd.Deterministic(name="p", loc=1.0/(1. + tf.exp(beta * temperature_ + alpha))).sample()

In [44]:
[
prior_alpha_,
    prior_beta_,
    p_deterministic_,
    D_,
] = evaluate([
    alpha,
    beta,
    p_deterministic,
    D,
])

NameError: name 'evaluate' is not defined